多输入输出通道

当输入包含多个通道时，需要构造一个与输入数据具有相同输入通道数的卷积核，以便与输入数据进行互相关运算。

由于输入和卷积核都有ci个通道，我们可以对每个通道输入的二维张量和卷积核的二维张量进行互相关运算，再对通道求和（将
的结果相加）得到二维张量。这是多通道输入和多输入通道卷积核之间进行二维互相关运算的结果。

简单来说就是对每个通道执行互相关操作，然后将结果相加。

In [3]:
import torch
from d2l import torch as d2l

def corr2d_multi_in(X, K):
    # 先遍历“X”和“K”的第0个维度（通道维度），再把它们加在一起
    return sum(d2l.corr2d(x, k) for x, k in zip(X, K))

构造输入张量和核张量来进行验证

In [4]:
X = torch.tensor([[[0.0, 1.0, 2.0], [3.0, 4.0, 5.0], [6.0, 7.0, 8.0]],
               [[1.0, 2.0, 3.0], [4.0, 5.0, 6.0], [7.0, 8.0, 9.0]]])
K = torch.tensor([[[0.0, 1.0], [2.0, 3.0]], [[1.0, 2.0], [3.0, 4.0]]])

corr2d_multi_in(X, K)

tensor([[ 56.,  72.],
        [104., 120.]])

目前编写的内容都只有单输出通道，但每一层有多个输出通道是至关重要的。

在最流行的神经网络架构中，随着神经网络层数的加深，我们常会增加输出通道的维数，通过减少空间分辨率以获得更大的通道深度。直观地说，我们可以将每个通道看作对不同特征的响应。
而现实可能更为复杂一些，因为每个通道不是独立学习的，而是为了共同使用而优化的。因此，多输出通道并不仅是学习多个单通道的检测器。

用ci和c0分别表示输入和输出通道的数目，并让kh和kw为卷积核的高度和宽度。为了获得多个通道的输出，我们可以为每个输出通道创建一个形状为ci*kh*kw的卷积核张量，这样卷积核的形状是c0*ci*kh*kw。
在互相关运算中，每个输出通道先获取所有输入通道，再以对应该输出通道的卷积核计算出结果。

In [5]:
# 首先计算一个多通道的输出的互相关函数
# X 是输入张量，形状为 (输入通道数, 高度, 宽度)，K 是卷积核张量，形状为 (输出通道数, 输入通道数, 核高度, 核宽度)
def corr2d_multi_in_out(X, K):
    # 迭代“K”的第0个维度，每次都对输入“X”执行互相关运算。
    # 最后将所有结果都叠加在一起
    return torch.stack([corr2d_multi_in(X, k) for k in K], 0) # 遍历K的第0个维度（输出通道维度），计算该通道的输出结果
    # 该过程相当于对于每个输出通道，使用对应的卷积核与输入的所有通道进行互相关运算并求和

K = torch.tensor([[[0.0, 1.0], [2.0, 3.0]], [[1.0, 2.0], [3.0, 4.0]]])
# 通过将核张量K与K+1(K中每个元素加1)和K+2连接起来，构造了一个具有3个输出通道的卷积核。
# 将所有输出通道的结果沿着第0维堆叠起来
# 原始的 K 形状是 (2, 2, 2)，表示 (输入通道数, 核高度, 核宽度)
K = torch.stack((K, K + 1, K + 2), 0)
print(K.shape)  # 这样最终输出的张量形状为 (输出通道数, 输出高度, 输出宽度)

# 下面，我们对输入张量X与卷积核张量K执行互相关运算。现在的输出包含3个通道，第一个通道的结果与先前输入张量X和多输入单输出通道的结果一致
corr2d_multi_in_out(X, K)

# 多通道输出的本质就是使用不同的卷积核对同一输入进行卷积操作
# 如果有c个输出通道，就会有c个不同的卷积核；每个卷积核负责提取输入数据中的不同特征

torch.Size([3, 2, 2, 2])


tensor([[[ 56.,  72.],
         [104., 120.]],

        [[ 76., 100.],
         [148., 172.]],

        [[ 96., 128.],
         [192., 224.]]])

1x1卷积层

卷积的本质是有效提取相邻像素间的相关特征，而1x1矩阵失去了该能力
实际上，1x1卷积唯一的计算发生在通道上，如调整通道数量，整合不同通道间的信息

正如上面提到的，不同输出通道的信息本质是用不同卷积核对所有输入通道进行卷积操作
因此1x1卷积将每个位置的不同输入通道下的元素进行了整合，一定程度上类似于全连接层，以ci个输入值转换为co个输出值，且保留了空间结构

In [6]:
def corr2d_multi_in_out_1x1(X, K):
    c_i, h, w = X.shape # 输入特征图的通道数、高度和宽度
    c_o = K.shape[0] # 输出通道数
    X = X.reshape((c_i, h * w)) # 将输入特征图展平为二维矩阵，形状为 (c_i, h * w)
    K = K.reshape((c_o, c_i)) # 将卷积核展平为二维矩阵，形状为 (c_o, c_i)
    # 于是相当于进行全连接层中的矩阵乘法
    # 每个输出通道的每个像素是不同输入通道下的线性组合
    Y = torch.matmul(K, X)
    return Y.reshape((c_o, h, w))

当执行1x1卷积运算时，上述函数相当于先前实现的互相关函数
下面的函数进行了验证

In [7]:
X = torch.normal(0, 1, (3, 3, 3))
K = torch.normal(0, 1, (2, 3, 1, 1))

Y1 = corr2d_multi_in_out_1x1(X, K)
Y2 = corr2d_multi_in_out(X, K)
assert float(torch.abs(Y1 - Y2).sum()) < 1e-6